In [1]:
# TODO - device a strategy to check the gradients provided by each of the main components of the sim.
# TODO - suggest starting with RWA environment (5 nodes, 4 slots per link) with incremental loading
# TODO - it might be advisable to start a new test file that contains all the gradient checks.
# TODO - implement a template test to check the gradients of a function
# TODO - need to consider any possible edges cases e.g. allocation failure/success, fully masked slots, etc.
# TODO - need to first of all check the gradients returned by each of the main component functions:
# implement
# check
# undo
# finalise
# generate_request
# get_reward_failure
# get_reward_success
# get_action_mask
# reset
# step
# heuristics
# TODO - testing the larger components should reveal issue with subfunctions e.g. get paths, get path slots, etc.

In [7]:
import jax
import time
import optax
import jax.numpy as jnp
from absl import flags
from flax.training.train_state import TrainState
from gymnax.environments import environment
from tensorflow_probability.substrates.jax.distributions.student_t import entropy

from xlron.environments.env_funcs import process_path_action
from xlron.environments.gn_model.isrs_gn_model import to_dbm
from xlron.environments.dataclasses import EnvState, EnvParams, VONETransition, RSATransition
from xlron.train.train_utils import *
import sys
from xlron.environments.make_env import *
from xlron import parameter_flags

# We load the default flags
FLAGS = flags.FLAGS

total_timesteps = 10

# Then define our experiment...
env_config = {
    "env_type": "rwa",
    "k": 3,
    "link_resources": 4,
    "incremental_loading" : True,
    "end_first_blocking": True,
    "topology_name": "5node_undirected",
    "values_bw": [1],
    "slot_size": 1,
    "max_requests": total_timesteps,
}

# ... and training details
train_config = {
    "SEED": 0,
    "NUM_LEARNERS": 1,
    "TOTAL_TIMESTEPS": total_timesteps,
    "NUM_ENVS": 1,
    "ROLLOUT_LENGTH": 5,
    "UPDATE_EPOCHS": 2,
    "LR": 5e-5,
    "GAMMA": 1.0, # For pre-training the VF, we don't use a discount factor
    "GAE_LAMBDA": 1.0, # For pre-training the VF, we don't use GAE
    "LR_SCHEDULE": "linear",
    "LAYER_NORM": True,
}

# We convert config to a Box container to make items accessible through both dict and dot notation
config = process_config(FLAGS, **env_config, **train_config)
config

Box({'logtostderr': False, 'alsologtostderr': False, 'log_dir': '', 'v': -1, 'verbosity': -1, 'logger_levels': {}, 'stderrthreshold': 'fatal', 'showprefixforinfo': True, 'chex_n_cpu_devices': 1, 'chex_assert_multiple_cpu_devices': False, 'run_with_pdb': False, 'pdb_post_mortem': False, 'pdb': False, 'run_with_profiling': False, 'profile_file': None, 'use_cprofile_for_profiling': True, 'only_check_args': False, 'test_srcdir': '', 'test_tmpdir': '/var/folders/5j/8r45ty_x4qj7wn99038f8m1c0000gn/T/absl_testing', 'test_random_seed': 301, 'test_randomize_ordering_seed': '', 'xml_output_file': '', 'chex_skip_pmap_variant_if_single_device': True, 'op_conversion_fallback_to_while_loop': True, 'delta_threshold': 0.5, 'tt_check_filter': False, 'tt_single_core_summaries': False, 'runtime_oom_exit': True, 'hbm_oom_exit': True, 'experimental_orbax_use_distributed_process_id': False, 'SEED': 0, 'NUM_LEARNERS': 1, 'NUM_DEVICES': 1, 'NUM_ENVS': 1, 'ROLLOUT_LENGTH': 5, 'NUM_UPDATES': 1, 'MINIBATCH_SIZE':

In [18]:
# INIT ENV
env, env_params = make(config)
rng = jax.random.PRNGKey(config.SEED)
rng, step_key, reset_key = jax.random.split(rng, 3)
obsv, env_state = env.reset(reset_key, env_params)

<class 'networkx.utils.decorators.argmap'> compilation 8:4: FutureWarning: laplacian_matrix will return a scipy.sparse array instead of a matrix in Networkx 3.0.


In [19]:
from gymnax.wrappers import GymnaxToGymWrapper
gym_env = GymnaxToGymWrapper(env._env, env_params)

Discrete(12)